# 📘 GVF Backwater Profile for a Trapezoidal Channel Upstream of a Weir

This notebook calculates the **smooth gradually varied flow (GVF)** profile starting from a weir for **trapezoidal channels**.

It also **exports results** into a downloadable **CSV file**.

---

## 📚 Governing Equations

**Cross-sectional Area:**
$$
A = b y + z y^2
$$

**Wetted Perimeter:**
$$
P = b + 2 y \sqrt{1+z^2}
$$

**Top Width:**
$$
T = b + 2 z y
$$

**Hydraulic Radius:**
$$
R = \frac{A}{P}
$$

**Critical Depth:** (solved from energy minimization)
$$
Q^2 T - g A^3 = 0
$$

**Normal Depth (Manning's Equation):**
$$
Q = \frac{1}{n} A R^{2/3} \sqrt{S_0}
$$

**Gradually Varied Flow (GVF) Equation:**
$$
\frac{dy}{dx} = \frac{S_0 - S_f}{1 - Fr^2}
$$
where:
- $$ S_f = \frac{n^2 Q^2}{A^2 R^{4/3}} $$
- $$ Fr = \frac{Q}{A \sqrt{g \frac{A}{T}}} $$

---

**Integration Approach:**
- Start slightly above critical depth.
- Integrate **upstream**.
- Step forward with small \( dx \).
- Stop after reaching desired reach length \( L \).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import fsolve
import ipywidgets as widgets
from IPython.display import FileLink, display
import os

g = 9.81
os.makedirs('outputs', exist_ok=True)

In [2]:
# Functions
def area(b, z, y):
    return b*y + z*y**2

def perimeter(b, z, y):
    return b + 2*y*(1 + z**2)**0.5

def top_width(b, z, y):
    return b + 2*z*y

def hydraulic_radius(b, z, y):
    A = area(b, z, y)
    P = perimeter(b, z, y)
    return A / P

def compute_critical_depth(Q, b, z):
    def func(y):
        A = area(b, z, y)
        T = top_width(b, z, y)
        return Q**2 * T - g * A**3
    return fsolve(func, 1.0)[0]

def Sf(Q, b, z, y, n):
    A = area(b, z, y)
    R = hydraulic_radius(b, z, y)
    return (n**2 * Q**2) / (A**2 * R**(4/3))

def Fr(Q, b, z, y):
    A = area(b, z, y)
    T = top_width(b, z, y)
    return Q / (A * np.sqrt(g * A / T))

In [3]:
def compute_gvf_profile(Q, b, z, n, S0, dx, L):
    y_crit = compute_critical_depth(Q, b, z)
    y_start = y_crit * 1.01

    x_vals = [0]
    y_vals = [y_start]
    x = 0
    y = y_start

    while x < L:
        fr = Fr(Q, b, z, y)
        denom = 1 - fr**2
        if abs(denom) < 1e-6:
            break
        dydx = (S0 - Sf(Q, b, z, y, n)) / denom
        y += dydx * dx
        x += dx
        if y <= 0 or np.isnan(y):
            break
        x_vals.append(x)
        y_vals.append(y)

    df = pd.DataFrame({
        'Distance (m)': x_vals,
        'Depth (m)': y_vals
    })
    csv_path = 'outputs/gvf_profile.csv'
    df.to_csv(csv_path, index=False)
    return x_vals, y_vals, y_crit, csv_path

In [4]:
def plot_gvf(Q, b, z, n, S0, dx, L):
    x_vals, y_vals, y_crit, csv_path = compute_gvf_profile(Q, b, z, n, S0, dx, L)

    plt.figure(figsize=(10, 5))
    plt.plot(x_vals, y_vals, label="Backwater Profile")
    plt.axhline(y_crit, color='r', linestyle='--', label="Critical Depth")
    plt.title(f"GVF Backwater Profile (Trapezoidal Channel)\nQ={Q} m³/s, b={b} m, z={z}")
    plt.xlabel("Distance upstream (m)")
    plt.ylabel("Water Depth (m)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    display(FileLink(csv_path, result_html_prefix="📄 Download results: "))

    print(f"Critical Depth: {y_crit:.3f} m")
    print(f"Water Depth at Upstream End: {y_vals[-1]:.3f} m")

In [5]:
widgets.interact(
    plot_gvf,
    Q=widgets.FloatSlider(value=10.0, min=1, max=30, step=1, description='Discharge Q'),
    b=widgets.FloatSlider(value=3.0, min=1, max=10, step=0.5, description='Bottom Width b'),
    z=widgets.FloatSlider(value=0.0, min=0, max=3, step=0.1, description='Side Slope z'),
    n=widgets.FloatSlider(value=0.03, min=0.01, max=0.1, step=0.005, description="Manning's n"),
    S0=widgets.FloatLogSlider(value=0.001, base=10, min=-4, max=-1, step=0.1, description='Slope S₀'),
    dx=widgets.FloatSlider(value=0.25, min=0.05, max=2.0, step=0.05, description='dx'),
    L=widgets.FloatSlider(value=100.0, min=10, max=300, step=10, description='Reach Length')
)

interactive(children=(FloatSlider(value=10.0, description='Discharge Q', max=30.0, min=1.0, step=1.0), FloatSl…

<function __main__.plot_gvf(Q, b, z, n, S0, dx, L)>